In [ ]:
!pip install firebase

In [ ]:
import pandas as pd
import requests
from firebase_admin import credentials, initialize_app, db

url = 'https://raw.githubusercontent.com/Ranash12/cloudCourse/main/Tirgul6/cloudtirgul6-firebase-adminsdk-3m2sb-5d58089424.json'
response = requests.get(url)
json_file = response.json()

# Set up Firebase connection
cred = credentials.Certificate(json_file)
initialize_app(cred, {'databaseURL': 'https://cloudtirgul6-default-rtdb.firebaseio.com/'})


In [24]:
# Function to fetch questions from Firebase
def fetch_questions():
    global questions_list
    questions_data = questions_ref.get()
    questions_list = [{'key': key, **value} for key, value in questions_data.items()]


In [45]:
from IPython.display import HTML, display

# Function to display main menu
def display_main_menu():
    clear_output(wait=True)
    
    def on_add_question_click(button):
        admin_input()

    def on_play_game_click(button):
        clear_output(wait=True)
        fetch_questions()
        display_game()

    title_html = '''
    <h2 style="text-align: center; margin-bottom: 20px;">Welcome to TrivPy Game</h2>
    '''
    display(HTML(title_html))
    
    add_question_button = widgets.Button(description="Add Question", layout=widgets.Layout(width='150px', height='auto'),)
    add_question_button.add_class('custom_button')
    add_question_button.add_class('add_question_button')
    add_question_button.on_click(on_add_question_click)
    play_game_button = widgets.Button(description="Start Game", layout=widgets.Layout(width='150px', height='auto'))
    play_game_button.add_class('custom_button')
    play_game_button.add_class('play_game_button')
    play_game_button.on_click(on_play_game_click)

    button_box = widgets.HBox([add_question_button, play_game_button], layout=widgets.Layout(justify_content='center'))
    display(button_box)

    style_html = '''
    <style>
        .custom_button {
            color: white;
            font-size: 1.1em;
            padding: 10px;
            border: none;
            cursor: pointer;
            margin-top: 10px;
        }
        .add_question_button {
            background-color: #4CAF50;
        }
        .play_game_button {
            background-color: #008CBA;
        }
    </style>
    '''
    display(HTML(style_html))


In [75]:
# Function to prompt admin for adding a question
def admin_input():
    clear_output(wait=True)

    display(HTML('<h2 style="text-align: center;">Enter the question and answers:</h2>'))

    question_input = widgets.Text(description="Question", layout=widgets.Layout(width='90%', margin='0 0 10px 0'))
    correct_answer_input = widgets.Text(description="Correct Answer", style={'description_width':'115px'}, layout=widgets.Layout(width='90%', margin='0 0 10px 0'))
    incorrect_answers_input = [widgets.Text(description=f"Incorrect Answer {i + 1}", style={'description_width':'115px'}, layout=widgets.Layout(width='90%', margin='0 0 10px 0')) for i in range(3)]
    submit_button = widgets.Button(description="Submit", button_style='success')
    cancel_button = widgets.Button(description="Cancel", button_style='danger')

    display(widgets.HBox([question_input], layout=widgets.Layout(justify_content='center')))
    display(widgets.HBox([correct_answer_input], layout=widgets.Layout(justify_content='center')))
    for answer_input in incorrect_answers_input:
        display(widgets.HBox([answer_input], layout=widgets.Layout(justify_content='center')))
    display(widgets.HBox([submit_button, cancel_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0')))

    def on_submit_click(button):
        question = question_input.value
        correct_answer = correct_answer_input.value
        answers = [answer_input.value for answer_input in incorrect_answers_input]
        add_question(question, correct_answer, *answers)
        display_main_menu()

    def on_cancel_click(button):
        display_main_menu()

    submit_button.on_click(on_submit_click)
    cancel_button.on_click(on_cancel_click)



# Function to add a question
def add_question(question, correct_answer, *answers):
    question_data = {
        'question': question,
        'correct_answer': correct_answer,
        'answers': [correct_answer, *answers]
    }
    questions_ref.push(question_data)

In [71]:
# Function to display game
def display_game():
    if questions_list:
        question_data = questions_list.pop(random.randint(0, len(questions_list) - 1))
        display_question(question_data)
    else:
        print(f"Game over! You scored {points} points.")
        print(f"Correct answers: {correct_answers}")
        print(f"Incorrect answers: {incorrect_answers}")
        print("Click the button below to restart the game.")
        restart_button = widgets.Button(description="Restart")
        display(restart_button)

        def on_restart_click(button):
            global questions_list, points, correct_answers, incorrect_answers
            clear_output(wait=True)
            points = 0
            correct_answers = 0
            incorrect_answers = 0
            fetch_questions()
            display_game()

        restart_button.on_click(on_restart_click)

# Function to display a question
def display_question(question_data):
    global points, correct_answers, incorrect_answers
    question = question_data['question']
    correct_answer = question_data['correct_answer']
    answers = question_data['answers']
    random.shuffle(answers)

    question_html = f'<h2 style="text-align: center;">{question}</h2>'
    display(HTML(question_html))
    
    buttons = [widgets.Button(description=answer, layout=widgets.Layout(width='auto')) for answer in answers]
    end_game_button = widgets.Button(description="End Game", button_style='danger')

    def on_button_click(button):
        global points, correct_answers, incorrect_answers
        if button.description == correct_answer:
            display(HTML('<h3 style="text-align: center; color: green;">Correct!</h3>'))
            points += 10
            correct_answers += 1
        else:
            display(HTML('<h3 style="text-align: center; color: red;">Incorrect!</h3>'))
            incorrect_answers += 1
        time.sleep(2.5)
        clear_output(wait=True)
        display_game()

    def on_end_game_click(button):
        clear_output(wait=True)
        display_end_game()

    buttons_box = widgets.HBox(buttons, layout=widgets.Layout(justify_content='center'))
    display(buttons_box)
    
    for button in buttons:
        button.on_click(on_button_click)
    
    end_game_button.on_click(on_end_game_click)
    end_game_button_box = widgets.HBox([end_game_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))
    display(end_game_button_box)




# Function to display end game results and pie chart
def display_end_game():
    display(HTML('<h2 style="text-align: center;">Game over!</h2>'))
    display(HTML(f'<h3 style="text-align: center;">You scored {points} points.</h3>'))
    display(HTML(f'<h3 style="text-align: center;">Correct answers: {correct_answers}</h3>'))
    display(HTML(f'<h3 style="text-align: center;">Incorrect answers: {incorrect_answers}</h3>'))

    if correct_answers + incorrect_answers > 0:
        labels = 'Correct', 'Incorrect'
        sizes = [correct_answers, incorrect_answers]
        colors = ['green', 'red']
        fig, ax = plt.subplots()
        ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        ax.axis('equal')
        plt.gca().set_position([0.5, 0.1, 0.8, 0.8])
        plt.show()
    else:
        display(HTML('<h3 style="text-align: center;">No questions were answered.</h3>'))
    
    display(HTML('<h3 style="text-align: center;">Click the button below to restart the game.</h3>'))
    restart_button = widgets.Button(description="Restart")
    restart_button.layout.margin = "0 auto"
    display(widgets.HBox([restart_button], layout=widgets.Layout(justify_content='center')))

    def on_restart_click(button):
        global questions_list, points, correct_answers, incorrect_answers
        clear_output(wait=True)
        points = 0
        correct_answers = 0
        incorrect_answers = 0
        fetch_questions()
        display_main_menu()

    restart_button.on_click(on_restart_click)



In [76]:
# Import necessary libraries
from IPython.display import display, clear_output
from IPython.display import HTML
import ipywidgets as widgets
import random
import time
import matplotlib.pyplot as plt

# Define global variables
questions_ref = db.reference('questions')
questions_list = []
points = 0
correct_answers = 0
incorrect_answers = 0

# Start the main menu
display_main_menu()